In [ ]:
using Distributions, Makie, Base.Threads, ColorSchemes
include.(("constant_simulations.jl", "psycho_sim_utils.jl"))

# Psychometric functions
In psychophysics we tend to try and find the boundary between two barely distinguishable events. That might be the absence vs presence of a stimulus or the exact change in stimulus parameter required to observe a difference. Signal detection theory treats all events as distributions from which the intensity can be sampled at a given point in time. The practical consequence of this is that when the stimulus is delivered on two (or N) separate occasions two (or N) different events will be perceived - depending on the sample from the distribution. To contextualize this in standard psychophysical tasks, in discrimination there is the standard stimulus with a given distribution. We then adjust a parameter in an attempt to shift the distribution of the comparison stimulus such that it no longer intersects with the distribution of the standard stimulus. Detection tasks are variants of this where the standard stimulus is no stimulus. When conceptualizing these distributions it is common to use an established uniform distribution (usually a normal/gaussian) and assume that all stimulus events represents samples from these distributions. For each stimulus level, the proportion of times the comparison stimulus is discriminated or the stimulus is detected represents the proportion of the distribution above outside the standard distribution or above the detection threshold.

As each stimulus level's distribution is assumed to be normal, and the difference between two normal distributions resembles a cumulative normal distribution, we can define a cumulative normal distribution with a mean and standard deviation (μ and σ) that represents the detection probability at each stimulus level. 